## Step 1: Import libraries

In [2]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import random
import tensorflow as tf

from scipy import ndimage
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report

from tensorflow.keras.layers import Conv3D, MaxPooling3D, Dense, \
    GlobalAveragePooling3D, BatchNormalization, Dropout, \
        UpSampling3D, concatenate, Flatten, ConvLSTM2D, Bidirectional, \
            Cropping3D, ZeroPadding3D, Activation, Input, UpSampling3D, \
                Conv3DTranspose
from tensorflow.keras.models import Sequential, save_model, load_model, Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

%matplotlib inline

## Step 2: Set Constants

In [3]:
SEED = 137
# using the range from segmentation
# [40:195,35:225,26:137]
# constants to crop images

MIN_WIDTH = 25 # 40
MAX_WIDTH = 215 # 195

MIN_HEIGHT = 35 # 35
MAX_HEIGHT = 225 # 225

MIN_DEPTH = 15 # 26
MAX_DEPTH = 143 # 137

In [4]:
#channel 
channel_1 = 't1'
channel_2 = 'seg'

## Step 3: Load Data

In [5]:
# load data
df_1 = pd.read_pickle(f"../../raw_data/Grade_ID_{channel_1}_nii.pkl")
df_2 = pd.read_pickle(f"../../raw_data/Grade_ID_{channel_2}_nii.pkl")
# the target columns have to be updated, it's only the fremework

#here we change the HGG to 1 and LGG to 0
df_1['Grade'] = df_1['Grade'].apply(lambda x: 1 if x == 'HGG' else 0)
df_2['Grade'] = df_2['Grade'].apply(lambda x: 1 if x == 'HGG' else 0)

## Step 4: EDA

In [6]:
print(f"Number of samples: {df_1[f'{channel_1}_nii'].shape[0]}")
print(f"Negative examples: {df_1[df_1['Grade'] == 0].shape}")
print(f"Positive examples: {df_1[df_1['Grade'] == 1].shape}")

print("")
print("Single example Shape:")
print(f"Height: {df_1[f'{channel_1}_nii'][0].shape[0]}")
print(f"Width: {df_1[f'{channel_1}_nii'][0].shape[1]}")
print(f"Depth: {df_1[f'{channel_1}_nii'][0].shape[2]}")

Number of samples: 369
Negative examples: (76, 3)
Positive examples: (293, 3)

Single example Shape:
Height: 240
Width: 240
Depth: 155


## Step 5: Crop Images and run Augmentation

In [7]:
# crop images
df_1[f'{channel_1}_nii'] = df_1[f'{channel_1}_nii'].apply(lambda x: np.array(x[MIN_HEIGHT:MAX_HEIGHT,MIN_WIDTH:MAX_WIDTH,MIN_DEPTH:MAX_DEPTH]))
X_1 = df_1[f'{channel_1}_nii']
df_2[f'{channel_2}_nii'] = df_2[f'{channel_2}_nii'].apply(lambda x: np.array(x[MIN_HEIGHT:MAX_HEIGHT,MIN_WIDTH:MAX_WIDTH,MIN_DEPTH:MAX_DEPTH]))
X_2 = df_2[f'{channel_2}_nii']
y = df_1['Grade']

In [8]:
# reshape input data

X_1 = np.array([np.array(val) for val in X_1])
X_2 = np.array([np.array(val) for val in X_2])

In [9]:
if channel_1 != 'seg':
    X_1[X_1>840]=840 # set the max value to 840
    X_1 = (X_1 - 0)/840 # Min-Max Scaler
if channel_2 != 'seg':
    X_2[X_2>840]=840 # set the max value to 840
    X_2 = (X_2 - 0)/840 # Min-Max Scaler
    

In [10]:
X_1 = X_1.reshape(len(X_1), X_1[0].shape[0], X_1[0].shape[1], X_1[0].shape[2],1)
X_2 = X_2.reshape(len(X_2), X_2[0].shape[0], X_2[0].shape[1], X_2[0].shape[2],1) # two channels


In [11]:
X =np.concatenate((X_1,X_2),axis=4)


In [12]:
X.shape

(369, 190, 190, 128, 2)

In [13]:
# create train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=SEED)
# reset the index of y, so that the indexes of X and y match
y_train = y_train.reset_index(drop=True)

y_test = y_test.reset_index(drop=True)

In [14]:
del [df_1,df_2,X_1,X_2,X]
import gc

gc.collect()

4

In [15]:
# find indices of positive examples
pos_indices = np.where(y_train == 0)[0]

# perform data augmentation on positive examples
rotated_examples1 = np.rot90(X_train[pos_indices], axes=(1, 2))
rotated_examples2 = np.rot90(rotated_examples1, axes=(1, 2))

# append new images and labels to the training set
X_train = np.concatenate([X_train, rotated_examples1, rotated_examples2], axis=0)
y_train = np.concatenate([y_train, np.repeat(y_train[pos_indices], 2)], axis=0)

In [16]:
np.unique(y_train,return_counts=True)

(array([0, 1]), array([183, 234]))

In [17]:
# conduct augmentation on X_train
# if not os.path.isfile(f"{channel_1}_{channel_2}-X_train.npy"):            
#     np.save(f"{channel_1}_{channel_2}-X_train.npy", X_train) #save the channel_1-_{channel_2}X so we don't need always augument the X
#     np.save(f"{channel_1}_{channel_2}-y_train.npy", y_train)
#     np.save(f"{channel_1}_{channel_2}-X_test.npy", X_test) 
#     np.save(f"{channel_1}_{channel_2}-y_test.npy", y_test)
# else:   #load the X direkly from npy file
#     X_train = np.load(f"{channel_1}_{channel_2}-X_train.npy",
#                     mmap_mode='r')
#     y_train = np.load(f"{channel_1}_{channel_2}-y_train.npy",
#                     mmap_mode='r')
#     X_test = np.load(f"{channel_1}_{channel_2}-X_test.npy",
#                     mmap_mode='r')
#     y_test = np.load(f"{channel_1}_{channel_2}-y_test.npy",
#                     mmap_mode='r')
    
    
    

In [18]:
X_train.shape

(417, 190, 190, 128, 2)

In [19]:
y_train.shape

(417,)

## Step 6: Define Model

In [20]:
##Sasha's but with sigmoid

in_layer = Input((190, 190, 128, 2)) #change the channel to 2
bn = BatchNormalization()(in_layer)
cn1 = Conv3D(8, 
             kernel_size = (1, 5, 5), 
             padding = 'same',
             activation = 'relu')(bn)
bn2 = Activation('relu')(BatchNormalization()(cn1))

dn1 = MaxPooling3D((2, 2, 2))(bn2)
cn3 = Conv3D(16, 
             kernel_size = (3, 3, 3),
             padding = 'same',
             activation = 'relu')(dn1)
bn3 = Activation('relu')(BatchNormalization()(cn3))

dn2 = MaxPooling3D((1, 2, 2))(bn3)
cn4 = Conv3D(32, 
             kernel_size = (3, 3, 3),
             padding = 'same',
             activation = 'relu')(dn2)
bn4 = Activation('relu')(BatchNormalization()(cn4))

up1 = Conv3DTranspose(16, 
                      kernel_size = (3, 3, 3),
                      strides = (1, 2, 2),
                     padding = 'same')(bn4)

cat1 = concatenate([up1, bn3], axis=2)

up2 = Conv3DTranspose(8, 
                      kernel_size = (3, 3, 3),
                      strides = (2, 2, 2),
                     padding = 'same')(cat1)

pre_out = concatenate([up2, bn2], axis=2)

#pre_out
pre_out = Conv3D(1, 
             kernel_size = (1, 1, 1), 
             padding = 'same',
             activation = 'relu')(pre_out)

#pre_out = Cropping3D((1, 2, 2))(pre_out) # avoid skewing boundaries
#out = ZeroPadding3D((1, 2, 2))(pre_out)
#pre_out = Dense(512, activation = 'relu')(pre_out)    

pre_out = Flatten()(pre_out)

pre_out = Dense(32, activation = 'relu')(pre_out) 
out = Dense(1, activation='sigmoid')(pre_out)
sim_model = Model(inputs = [in_layer], outputs = [out])
sim_model.summary()

2023-03-15 09:26:10.227326: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-15 09:26:10.227373: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-15 09:26:10.227389: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (brain-project-cpu): /proc/driver/nvidia/version does not exist
2023-03-15 09:26:10.227615: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 190, 190, 1  0           []                               
                                28, 2)]                                                           
                                                                                                  
 batch_normalization (BatchNorm  (None, 190, 190, 12  8          ['input_1[0][0]']                
 alization)                     8, 2)                                                             
                                                                                                  
 conv3d (Conv3D)                (None, 190, 190, 12  408         ['batch_normalization[0][0]']    
                                8, 8)                                                         

In [21]:
optim=Adam(learning_rate= 0.0001)
sim_model.compile(loss = 'binary_crossentropy',
                  optimizer = optim,
                  metrics = ['accuracy'])

In [22]:
print(f" Size of X_train: {(X_train.size * X_train.itemsize) / 1e9} Gb")

 Size of X_train: 30.8299776 Gb


## Step 7: Train Model

In [23]:
es = EarlyStopping(patience=5, restore_best_weights = True)
history = sim_model.fit(X_train, y_train,
                        epochs = 30,
                        batch_size = 4,
                        callbacks = [es],
                        validation_data=(X_test, y_test),
                        shuffle =True,
                        verbose = 1)
sim_model.save(f'model_glioma_{channel_1}_{channel_2}_nii_3dUnet')

Epoch 1/30
105/105 [==============================] - 259s 2s/step - loss: 9.6847 - accuracy: 0.7914 - val_loss: 0.5412 - val_accuracy: 0.8108
Epoch 2/30
105/105 [==============================] - 253s 2s/step - loss: 0.2869 - accuracy: 0.8609 - val_loss: 0.5355 - val_accuracy: 0.7973
Epoch 3/30
 53/105 [==============>...............] - ETA: 2:01 - loss: 0.2643 - accuracy: 0.8349

KeyboardInterrupt: 

In [ ]:
#plot the learning curve
def plot_loss(history):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(13,4))
    ax1.plot(history.history['loss'])
    ax1.plot(history.history['val_loss'])
    ax1.set_title('Model loss')
    ax1.set_ylabel('Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylim(ymin=0, ymax=5)
    ax1.legend(['Train', 'Validation'], loc='best')
    ax1.grid(axis="x",linewidth=0.2)
    ax1.grid(axis="y",linewidth=0.2)    
    
    ax2.plot(history.history['accuracy'])
    ax2.plot(history.history['val_accuracy'])
    ax2.set_title('Accuracy')
    ax2.set_ylabel('Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylim(ymin=0, ymax=1)
    ax2.legend(['Train', 'Validation'], loc='best')
    ax2.grid(axis="x",linewidth=0.2)
    ax2.grid(axis="y",linewidth=0.2)    

    plt.show()    

In [ ]:
plot_loss(history)

In [ ]:
predictions = (sim_model.predict(X_test) > 0.5).astype("int32")
predictions = pd.DataFrame(predictions)


target_names = ['class 0', 'class 1']
print(classification_report(y_test, predictions, target_names=target_names))

In [ ]:


confusion_matrix = metrics.confusion_matrix(y_test, predictions)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()